## Setting LLM Model

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
#print("Open AI - ",os.getenv("LITELLM_URL"),os.getenv("OPENAI_API_MODEL"), os.getenv("OPENAI_API_EMBEDDING"))
#print("OLLAMA  - ",os.getenv("OLLAMA_URL"),os.getenv("OLLAMA_MODEL"))
#print("Local OLLAMA - ",os.getenv("OLLAMA_LOCAL_URL"),os.getenv("OLLAMA_LOCAL_MODEL"))

True

In [3]:
from llama_index.core import Settings

### RUN LLM AS OLLAMA 

In [ ]:
#install OPEN AI LLM, skip if already installed
!pipenv install llama-index-llms-ollama

In [5]:
from llama_index.llms.ollama import Ollama
api_base=os.getenv("OLLAMA_LOCAL_URL")
model=os.getenv("OLLAMA_LOCAL_MODEL")
llm = Ollama(model=model, base_url=api_base,request_timeout=120.0)

# use remote ollam
"""
api_base=os.getenv("OLLAMA_URL")
model=os.getenv("OLLAMA_MODEL")
llm = Ollama(model=model, base_url=api_base,request_timeout=180.0)
"""
#test run
#response = llm.complete("What is the capital of France?")
#print(response)

'\napi_base=os.getenv("OLLAMA_URL")\nmodel=os.getenv("OLLAMA_MODEL")\nllm = Ollama(model=model, base_url=api_base,request_timeout=180.0)\n'

### RUN LLM AS OPEN AI 

In [ ]:
#install OPEN AI LLM, skip if already installed
!pipenv install llama-index-llms-openai

In [4]:
from llama_index.llms.openai import OpenAI
api_base=os.getenv("LITELLM_URL")
model=os.getenv("OPENAI_API_MODEL")

Settings.llm = OpenAI(
    model=model,
    api_base = api_base,
    temperature=0.3
)

#resp = Settings.llm.complete("What is the capital of France?")
#print(resp)

## Setting Embedding Model

In [5]:
# use open AI embedding
from llama_index.embeddings.openai import OpenAIEmbedding
api_base=os.getenv("LITELLM_URL")
embedding_model=os.getenv("OPENAI_API_EMBEDDING")

Settings.embed_model = OpenAIEmbedding(
    model_name=embedding_model,
    api_base = api_base,
)
#embed_text = Settings.embed_model.get_text_embedding("hello")
#print(f"{len(embed_text)}, {embed_text}")

In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("./data/paul").load_data()

In [7]:
import logging
import sys

#logging.basicConfig(stream=sys.stdout, level=logging.INFO)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from IPython.display import Markdown, display

### Node
Nodes represent "chunks" of source Documents, whether that is a text chunk, an image, or more. They also contain metadata and relationship information with other nodes and index structures

In [8]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
nodes = parser.get_nodes_from_documents(documents)

In [9]:
print(f"Total Nodes: {len(nodes)}")
for node in nodes[:5]:
    print(f"Node ID: {node.node_id}, {node.text[0:50]}")

Total Nodes: 18
Node ID: 856378fb-778e-41f8-8a48-47684b3734be, What I Worked On

February 2021

Before college th
Node ID: 145e55c0-8029-42c8-9848-26f44cad73fc, It seemed only a matter of time before we'd have M
Node ID: 4a851ab0-7b85-4d5d-8131-6c260f2dd1fe, There were some surplus Xerox Dandelions floating 
Node ID: 81ae6a47-ead3-436b-b5a1-085181b72f44, This was now only weeks away. My nice landlady let
Node ID: 1d671e3b-6f6c-4d05-99dd-3ff60e2313e7, Our teacher, professor Ulivi, was a nice guy. He c


### HierarchicalNodeParser && AutoMergingRetriever
AutoMergingRetriever, which looks at a set of leaf nodes and recursively "merges" subsets of leaf nodes that reference a parent node beyond a given threshold. This allows us to consolidate potentially disparate, smaller contexts into a larger context that might help synthesis.

In [10]:
from llama_index.core import Document

In [11]:
doc_text = "\n\n".join([d.get_content() for d in documents])
docs = [Document(text=doc_text)]

print(len(docs),docs[0].text[100:500])

1 were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing


In [12]:
from llama_index.core.node_parser import HierarchicalNodeParser
node_parser = HierarchicalNodeParser.from_defaults()
nodes = node_parser.get_nodes_from_documents(docs)

In [13]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

leaf_nodes = get_leaf_nodes(nodes)
len(leaf_nodes)

176

In [14]:
root_nodes = get_root_nodes(nodes)
len(root_nodes)

9

In [15]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext

docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)

In [16]:
## Load index into vector index
from llama_index.core import VectorStoreIndex

base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
)

In [17]:
from llama_index.core.retrievers import AutoMergingRetriever
base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)

In [18]:
query_str = (
    "Who is Paul Graha?"
)

nodes = retriever.retrieve(query_str)
base_nodes = base_retriever.retrieve(query_str)

In [19]:
len(nodes)

6

In [20]:
len(base_nodes)

6

In [21]:
from llama_index.core.response.notebook_utils import display_source_node

for node in nodes:
    display_source_node(node, source_length=10000)

**Node ID:** ccd2fd92-6b24-4a3b-8058-abdfef977896<br>**Similarity:** 0.1910078956684135<br>**Text:** The following spring, lightning struck. I was invited to give a talk at a Lisp conference, so I gave one about how we'd used Lisp at Viaweb. Afterward I put a postscript file of this talk online, on paulgraham.com, which I'd created years before using Viaweb but had never used for anything. In one day it got 30,000 page views. What on earth had happened? The referring urls showed that someone had posted it on Slashdot. [10]

Wow, I thought, there's an audience.<br>

**Node ID:** ea41480e-5b76-4bf1-9086-623873c35cf6<br>**Similarity:** 0.18024639939898324<br>**Text:** They were an impressive group. That first batch included reddit, Justin Kan and Emmett Shear, who went on to found Twitch, Aaron Swartz, who had already helped write the RSS spec and would a few years later become a martyr for open access, and Sam Altman, who would later become the second president of YC. I don't think it was entirely luck that the first batch was so good. You had to be pretty bold to sign up for a weird thing like the Summer Founders Program instead of a summer job at a legit place like Microsoft or Goldman Sachs.<br>

**Node ID:** ed821ea3-e039-44e9-917e-3a7a4b6524fa<br>**Similarity:** 0.17983075248980943<br>**Text:** But I told few people about Bel while I was working on it. So for years it must have seemed that I was doing nothing, when in fact I was working harder than I'd ever worked on anything. Occasionally after wrestling for hours with some gruesome bug I'd check Twitter or HN and see someone asking "Does Paul Graham still code?"

Working on Bel was hard but satisfying.<br>

**Node ID:** 78c4e181-adc2-4433-8655-b7b5ac0a73b7<br>**Similarity:** 0.17158860607657678<br>**Text:** That fall I started taking art classes at Harvard. Grad students could take classes in any department, and my advisor, Tom Cheatham, was very easy going. If he even knew about the strange classes I was taking, he never said anything.

So now I was in a PhD program in computer science, yet planning to be an artist, yet also genuinely in love with Lisp hacking and working away at On Lisp. In other words, like many a grad student, I was working energetically on multiple projects that were not my thesis.

I didn't see a way out of this situation.<br>

**Node ID:** 0a8a79ec-2d97-4a0c-bee0-7ed550b40b2d<br>**Similarity:** 0.171288554992616<br>**Text:** Three separate hosts would all invite their friends to one party. So for every guest, two thirds of the other guests would be people they didn't know but would probably like. One of the guests was someone I didn't know but would turn out to like a lot: a woman called Jessica Livingston. A couple days later I asked her out.

Jessica was in charge of marketing at a Boston investment bank. This bank thought it understood startups, but over the next year, as she met friends of mine from the startup world, she was surprised how different reality was. And how colorful their stories were.<br>

**Node ID:** eb745a47-c9aa-4e5c-804d-f510924d4549<br>**Similarity:** 0.1663666158376561<br>**Text:** I also worked on spam filters, and did some more painting. I used to have dinners for a group of friends every thursday night, which taught me how to cook for groups. And I bought another building in Cambridge, a former candy factory (and later, twas said, porn studio), to use as an office.

One night in October 2003 there was a big party at my house. It was a clever idea of my friend Maria Daniels, who was one of the thursday diners. Three separate hosts would all invite their friends to one party.<br>

In [22]:
for node in base_nodes:
    display_source_node(node, source_length=10000)

**Node ID:** ccd2fd92-6b24-4a3b-8058-abdfef977896<br>**Similarity:** 0.1910078956684135<br>**Text:** The following spring, lightning struck. I was invited to give a talk at a Lisp conference, so I gave one about how we'd used Lisp at Viaweb. Afterward I put a postscript file of this talk online, on paulgraham.com, which I'd created years before using Viaweb but had never used for anything. In one day it got 30,000 page views. What on earth had happened? The referring urls showed that someone had posted it on Slashdot. [10]

Wow, I thought, there's an audience.<br>

**Node ID:** ea41480e-5b76-4bf1-9086-623873c35cf6<br>**Similarity:** 0.18024639939898324<br>**Text:** They were an impressive group. That first batch included reddit, Justin Kan and Emmett Shear, who went on to found Twitch, Aaron Swartz, who had already helped write the RSS spec and would a few years later become a martyr for open access, and Sam Altman, who would later become the second president of YC. I don't think it was entirely luck that the first batch was so good. You had to be pretty bold to sign up for a weird thing like the Summer Founders Program instead of a summer job at a legit place like Microsoft or Goldman Sachs.<br>

**Node ID:** ed821ea3-e039-44e9-917e-3a7a4b6524fa<br>**Similarity:** 0.17983075248980943<br>**Text:** But I told few people about Bel while I was working on it. So for years it must have seemed that I was doing nothing, when in fact I was working harder than I'd ever worked on anything. Occasionally after wrestling for hours with some gruesome bug I'd check Twitter or HN and see someone asking "Does Paul Graham still code?"

Working on Bel was hard but satisfying.<br>

**Node ID:** 78c4e181-adc2-4433-8655-b7b5ac0a73b7<br>**Similarity:** 0.17158860607657678<br>**Text:** That fall I started taking art classes at Harvard. Grad students could take classes in any department, and my advisor, Tom Cheatham, was very easy going. If he even knew about the strange classes I was taking, he never said anything.

So now I was in a PhD program in computer science, yet planning to be an artist, yet also genuinely in love with Lisp hacking and working away at On Lisp. In other words, like many a grad student, I was working energetically on multiple projects that were not my thesis.

I didn't see a way out of this situation.<br>

**Node ID:** 0a8a79ec-2d97-4a0c-bee0-7ed550b40b2d<br>**Similarity:** 0.171288554992616<br>**Text:** Three separate hosts would all invite their friends to one party. So for every guest, two thirds of the other guests would be people they didn't know but would probably like. One of the guests was someone I didn't know but would turn out to like a lot: a woman called Jessica Livingston. A couple days later I asked her out.

Jessica was in charge of marketing at a Boston investment bank. This bank thought it understood startups, but over the next year, as she met friends of mine from the startup world, she was surprised how different reality was. And how colorful their stories were.<br>

**Node ID:** eb745a47-c9aa-4e5c-804d-f510924d4549<br>**Similarity:** 0.1663666158376561<br>**Text:** I also worked on spam filters, and did some more painting. I used to have dinners for a group of friends every thursday night, which taught me how to cook for groups. And I bought another building in Cambridge, a former candy factory (and later, twas said, porn studio), to use as an office.

One night in October 2003 there was a big party at my house. It was a clever idea of my friend Maria Daniels, who was one of the thursday diners. Three separate hosts would all invite their friends to one party.<br>

In [23]:
from llama_index.core.query_engine import RetrieverQueryEngine

In [24]:
query_engine = RetrieverQueryEngine.from_args(retriever)
base_query_engine = RetrieverQueryEngine.from_args(base_retriever)

In [25]:
response = query_engine.query(query_str)
print(response)

Paul Graham is a computer scientist, entrepreneur, and writer known for his work with Lisp programming and for founding Viaweb, one of the first web-based applications. He has been involved in various projects, including the Summer Founders Program and has contributed to the startup community. Additionally, he has written extensively about technology and entrepreneurship.


In [26]:
base_response = base_query_engine.query(query_str)
print(base_response)

Paul Graham is a computer scientist and entrepreneur known for his work with Lisp programming and for co-founding the startup incubator Y Combinator (YC). He has a background in both computer science and art, and he has been involved in various projects, including the development of spam filters and the writing of the book "On Lisp." Additionally, he has had a significant impact on the startup community, connecting with influential figures in the tech industry.
